# Structured Output Parser

<b>`Structured Output Parser`</b> is an output parser in LangChain that helps extract structured JSON data from LLM responses based on predefined field schemas.

It works by defining a list of fields (Response Schema) that the model should return,ensuring the output follows a structured format

In [1]:
# from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

In [2]:
load_dotenv()

True

In [3]:
API_KEY = os.environ['GEMINI_API_KEY']
MODEL_NAME = 'gemini-2.0-flash'

In [4]:
model = ChatGoogleGenerativeAI(api_key=API_KEY, model=MODEL_NAME)

In [5]:
schema = [
    ResponseSchema(name='fact_1', description="Fact 1 about the topic"),
    ResponseSchema(name='fact_2', description="Fact 2 about the topic"),
    ResponseSchema(name='fact_3', description="Fact 3 about the topic"),
]

In [6]:
schema

[ResponseSchema(name='fact_1', description='Fact 1 about the topic', type='string'),
 ResponseSchema(name='fact_2', description='Fact 2 about the topic', type='string'),
 ResponseSchema(name='fact_3', description='Fact 3 about the topic', type='string')]

In [7]:
parser = StructuredOutputParser.from_response_schemas(schema)

In [8]:
parser

StructuredOutputParser(response_schemas=[ResponseSchema(name='fact_1', description='Fact 1 about the topic', type='string'), ResponseSchema(name='fact_2', description='Fact 2 about the topic', type='string'), ResponseSchema(name='fact_3', description='Fact 3 about the topic', type='string')])

In [9]:
template = PromptTemplate(
    template="Write fact about top 3 Character with their name from the game {game_name}  \n {format_instruction}",
    input_variable=['game_name'],
    partial_variables={"format_instruction": parser.get_format_instructions()}
)

In [10]:
template

PromptTemplate(input_variables=['game_name'], input_types={}, partial_variables={'format_instruction': 'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"fact_1": string  // Fact 1 about the topic\n\t"fact_2": string  // Fact 2 about the topic\n\t"fact_3": string  // Fact 3 about the topic\n}\n```'}, template='Write fact about top 3 Character with their name from the game {game_name}  \n {format_instruction}')

In [11]:
prompt = template.format(game_name= "Red Death Redemption 2")

In [12]:
prompt

'Write fact about top 3 Character with their name from the game Red Death Redemption 2  \n The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"fact_1": string  // Fact 1 about the topic\n\t"fact_2": string  // Fact 2 about the topic\n\t"fact_3": string  // Fact 3 about the topic\n}\n```'

In [13]:
result = model.invoke(prompt)

In [14]:
result

AIMessage(content='```json\n{\n\t"fact_1": "Arthur Morgan: Despite being an outlaw, Arthur possesses a strong moral compass and grapples with the consequences of his actions, especially as he confronts his own mortality due to tuberculosis.",\n\t"fact_2": "John Marston: John\'s dedication to his family and desire to leave his outlaw past behind him are constantly tested by the lingering influence of the Van der Linde gang and the encroaching forces of civilization.",\n\t"fact_3": "Dutch van der Linde: Dutch\'s charismatic leadership and belief in a utopian vision gradually deteriorate throughout the game, leading him to increasingly erratic and violent decisions as his control over the gang slips away."\n}\n```', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--4af55d31-ecd1-4adb-827c-a7a2af447ccc-0')

In [15]:
final_result = parser.parse(result.content)

In [16]:
final_result

{'fact_1': 'Arthur Morgan: Despite being an outlaw, Arthur possesses a strong moral compass and grapples with the consequences of his actions, especially as he confronts his own mortality due to tuberculosis.',
 'fact_2': "John Marston: John's dedication to his family and desire to leave his outlaw past behind him are constantly tested by the lingering influence of the Van der Linde gang and the encroaching forces of civilization.",
 'fact_3': "Dutch van der Linde: Dutch's charismatic leadership and belief in a utopian vision gradually deteriorate throughout the game, leading him to increasingly erratic and violent decisions as his control over the gang slips away."}

# <center> Doing this same code using `chain`

In [17]:
import os 
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain_core.prompts import PromptTemplate

In [18]:
API_KEY = os.environ['GEMINI_API_KEY']
MODEL_NAME = 'gemini-2.0-flash'

In [19]:
model = ChatGoogleGenerativeAI(api_key=API_KEY, model=MODEL_NAME)

In [20]:
schema = [
    ResponseSchema(name="name", description="Write Name of the Character"),
    ResponseSchema(name="age", description="Write Age of the Character"),
    ResponseSchema(name="post", description="Write Post or designation of the Character"),
    ResponseSchema(name="wife", description="Write wife name of the Character"),
]

In [21]:
schema

[ResponseSchema(name='name', description='Write Name of the Character', type='string'),
 ResponseSchema(name='age', description='Write Age of the Character', type='string'),
 ResponseSchema(name='post', description='Write Post or designation of the Character', type='string'),
 ResponseSchema(name='wife', description='Write wife name of the Character', type='string')]

In [22]:
parser = StructuredOutputParser.from_response_schemas(schema)

In [23]:
template = PromptTemplate(
    template="Write me about main of the character Details of game {game_name} \n {format_instructions}",
    input_variables=['game_name'],
    partial_variables={'format_instructions': parser.get_format_instructions()}
)

In [24]:
template

PromptTemplate(input_variables=['game_name'], input_types={}, partial_variables={'format_instructions': 'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"name": string  // Write Name of the Character\n\t"age": string  // Write Age of the Character\n\t"post": string  // Write Post or designation of the Character\n\t"wife": string  // Write wife name of the Character\n}\n```'}, template='Write me about main of the character Details of game {game_name} \n {format_instructions}')

In [25]:
chain = template | model | parser

In [26]:
result = chain.invoke({"game_name": "Red dead redemption 2"})

In [27]:
result

{'name': 'Arthur Morgan',
 'age': '36 (in 1899)',
 'post': 'Enforcer and main protagonist of the Van der Linde gang',
 'wife': 'Deceased; Mary Gillis (former lover, not wife)'}